# Importing ODV aggregated spreadsheet data

* Data are supposed to be aggregated using ODV and exported as aggregated ODV spreadsheet or ODV netCDF files.
* Substitute the file name `small_ODV_sample.txt` with the file name of your aggregated ODV file. 
* Do not export "data error" from ODV (column header `STANDARD_DEV`).

This notebook describes the reading of
1. ODV spreadsheet and 
2. ODV netCDF files.

<div class="alert alert-block alert-info">
ℹ️ This notebook shows how to read ODV spreadsheet and netCDF files.<br>
The latter format is nevertheless preferred, as it allows a faster reading, especially for big files.
</div>

In [ ]:
using DIVAnd
using Makie, CairoMakie, GeoMakie
using NCDatasets
using Dates
using Statistics
using DelimitedFiles
include("../config.jl")

## 1. Read spreadsheet file
Substitute `small_ODV_sample.txt` with the file name of your aggregated ODV file.     
Or you can download the small test file (71 stations).

In [ ]:
download_check(smallODVfile, smallODVfileURL)

Aggregated ODV files do not have a semantic header, therefore we need to extract the corresponding column by using the "local" column header name (instead of being able to use the P01 name for the ODV files conforming to 
[Specification of SeaDataNet Data Transport Formats](https://www.seadatanet.org/content/download/636/3333/file/SDN2_D85_WP8_Datafile_formats.pdf?version=2)).


By default only `good` and `probably good` values are loaded.     
This can be changed using the optional parameter `qv_flags`:

In [ ]:
DIVAnd.ODVspreadsheet.GOOD_VALUE

In [ ]:
?ODVspreadsheet.load

If for some reasons, the column name contains an underscore (`Water_body_phosphate` as opposed to `Water body phosphate`), then the local name should also use underscoes.

In [ ]:
obsval, obslon, obslat, obsdepth, obstime, obsid = ODVspreadsheet.load(
    Float64,
    [smallODVfile],
    ["Water body salinity"];
    nametype = :localname,
);

Basic range check for the data (and presence of NaN and Inf)

In [ ]:
checkobs((obslon, obslat, obsdepth, obstime), obsval, obsid)

Individual elements can be retrieved by indexing `obsdata`, `obslat`, `obslon`, `obsdepth` and `obstime`,      
for example:

In [ ]:
obsval[10]

is the measurement (salinity here) of the 10th data point.

### Quick plot
To see the data locations

In [ ]:
f = CairoMakie.Figure()
ax = Axis(f[1, 1], title = "Observations")
CairoMakie.scatter!(ax, obslon, obslat)
f

### Remove data from the file
Generate a text file to keep track of the removed data.        
Define the indices of the data to delete:

In [ ]:
index = [10, 14]

Create an array containing these data:

In [ ]:
baddata = [
    "lon" "lat" "depth" "time" "value" "ids"
    obslon[index] obslat[index] obsdepth[index] obstime[index] obsval[index] obsid[index]
]

The array will be written to a file as a text using the function `writedlm`.

In [ ]:
?writedlm

In [ ]:
sel = trues(size(obslon))
sel[index] .= false

obslon_only_good_data = obslon[sel];
obslat_only_good_data = obslat[sel];
obsdepth_only_good_data = obsdepth[sel];
obstime_only_good_data = obstime[sel];
obsdata_only_good_data = obsval[sel];
obsids_only_good_data = obsid[sel];

@show size(obslon_only_good_data);
@show size(obslon);

In [ ]:
writedlm(joinpath(datadir, "my_bad_data.txt"), baddata)

The identifier is a combination of the EDMO code and LOCAL CDI ID

In [ ]:
;cat ../data/my_bad_data.txt

## Select data according to criterion
For the purpose of the example, let's assume we want to keep only the salinity values below 39.25 (even if the observations are good).

In [ ]:
sel = obsval .< 39.25;

index = findall(.!sel)
@info("Number of removed observations: $(length(index))");

obsval_good = obsval[sel]
obslon_good = obslon[sel]
obslat_good = obslat[sel]
obsdepth_good = obsdepth[sel]
obstime_good = obstime[sel]
obsid_good = obsid[sel];

In [ ]:
checkobs((obslon_good, obslat_good, obsdepth_good, obstime_good), obsval_good, obsid_good)

Here we use a criterion based on the depth and on the time of measurement (month):

In [ ]:
sel = (obsdepth .< 50.0) .& (Dates.month.(obstime) .== 10)
@show sum(sel);
@show length(obsval);
obsval_new = obsval[sel];

Let's create a histogram showing the number of observations per month:

In [ ]:
f = Figure()
ax = Axis(f[1, 1], title = "Number of observations\nper monthly period")
hist!(ax, Dates.month.(obstime), bins = 12)
f

### Plot the selected data positions

In [ ]:
bathname = gebco16file
download_check(gebco16file, gebco16fileURL)
bathisglobal = true

# Extract the bathymetry for plotting
lonr = extrema(obslon[sel])
latr = extrema(obslat[sel])

lonr = -10.0:30.0
latr = 30.0:45.0
bx, by, b = extract_bath(bathname, bathisglobal, lonr, latr);

In [ ]:
using GeoMakie

In [ ]:
GeoMakie

In [ ]:
f = Figure()
ax = GeoAxis(f[1,1], title="Selected data points")
contourf!(ax, bx, by, b, levels = 2, colormap=Reverse("binary"))
scatter!(ax, obslon[sel],obslat[sel]; color = obsval[sel])
f

## 2. Read ODV netCDF
Download the file or use one of your file.

In [ ]:
download_check(smallODVncfile, smallODVncfileURL)

The function `NCODV.load` is designed to read netCDF files generated by ODV. In addition it has been optimised so large files (a few GB) can be ingested in a few minutes.

In [ ]:
@time obsval2, obslon2, obslat2, obsdepth2, obstime2, obsid2 =
    NCODV.load(Float64, smallODVncfile, "Water body salinity");

A quick plot helps to check if we obtain the same data as with the spreadsheet (1st figure of the notebook):

In [ ]:
f = Figure()
ax = Axis(f[1, 1], title = "Observations")
scatter!(ax, obslon2, obslat2)
f

<div class="alert alert-block alert-info">
<b>Note:</b> when you expect to have to read several times a large netCDF file, it is convenient to save it in a simplified form, easier and faster to read than the ODV netCDF file.           
This can be done using the function <code>saveobs</code>.
</div> 

In [ ]:
datafile3 = joinpath(datadir, "small_ODV_sample_simple.nc")
isfile(datafile3) ? rm(datafile3) : @debug("ok")
DIVAnd.saveobs(
    datafile3,
    "salinity",
    obsval2,
    (obslon2, obslat2, obsdepth2, obstime2),
    obsid2,
)

That file can then be read using the function `loadobs`.

In [ ]:
@time obsval3, obslon3, obslat3, obsdepth3, obstime3, obsids3 =
    loadobs(Float64, datafile3, "salinity");